In [1]:
import requests, datetime, pickle,tqdm

URL = 'https://open-api.bingx.com'

descargar_data = True

# Obtener la serie de precios para BTC, ETH y XRP 

En temporalidades: 1d y/o 4h, 2h, 1h, 30m, 15m

data_raw = {<br />
    'BTC': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'ETH': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'XRP':<br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    }<br />
}<br />

In [2]:
from backtesting.fechas import past_timestamp, now_timestamp, retrasar_tres_horas
from backtesting.data import response_to_dataframe
from backtesting.ficheros import crear_directorio

In [3]:
if not descargar_data:
    with open('./data/data_raw.pickle', 'rb') as f:
        data_raw = pickle.load(f)
    activos = list(data_raw.keys())
    temporalidades = list(data_raw[activos[0]].keys())

else:
    path = '/openApi/swap/v2/quote/klines'

    data_1d = True
    data_low_temporality = True

    temporalidades = []
    if data_1d:
        temporalidades.append('1d')
    if data_low_temporality:
        temporalidades.append('4h')
        temporalidades.append('2h')
        temporalidades.append('1h')
        temporalidades.append('30m')
        temporalidades.append('15m')
    if not data_1d and not data_low_temporality:
        exit()

    now = datetime.datetime.now()
    start_time = []
    for t in temporalidades:
        if t == '1d':
            start_time.append(int(past_timestamp(365, 'days', now)))
        elif t == '4h':
            start_time.append(int(past_timestamp(45, 'days', now)))
        elif t == '2h':
            start_time.append(int(past_timestamp(20, 'days', now)))
        elif t == '1h':
            start_time.append(int(past_timestamp(20, 'days', now)))
        elif t == '30m':
            start_time.append(int(past_timestamp(6, 'days', now)))
        elif t == '15m':
            start_time.append(int(past_timestamp(3, 'days', now)))

    limit = 555
    activos = ['BTC', 'ETH', 'XRP', 'ENS']
    service = URL + path
    endTime = int(datetime.datetime.timestamp(datetime.datetime.now()) * 1000)
    data_raw = {}
    for activo in activos:
        data_raw[activo] = {}
        symbol = activo + '-USDT'
        for i, interval in enumerate(temporalidades):
            response = requests.get(service, params={'symbol':symbol, 'interval':interval, 'limit':limit, 'startTime':start_time[i], 'endTime':endTime})
            df = response_to_dataframe(response.json())
            if df is None:
                continue
            df['time'] = df['time'].apply(retrasar_tres_horas)
            df = df.set_index('time')
            data_raw[activo][interval] = df

    crear_directorio('data')
    with open('./data/data_raw.pickle', 'wb') as f:
        pickle.dump(data_raw, f, pickle.HIGHEST_PROTOCOL)

# Aplicar Squeeze Momentum Indicator a cada serie temporal y obtener la temporalidad que tengan alta correlacion para cada activo

SMI_default = {<br />
    'BTC': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'ETH': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'XRP':<br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    }<br />
<br />
}<br />
<br />
SMI_default_corr = {<br />
    'BTC': <br />
    {'1d': 0, '4h': 0, '2h': 0, '1h': 0, '30m': 0, '15m': 0<br />
    },<br />
    'ETH': <br />
    {'1d': 0, '4h': 0, '2h': 0, '1h': 0, '30m': 0, '15m': 0<br />
    },<br />
    'XRP':<br />
    {'1d': 0, '4h': 0, '2h': 0, '1h': 0, '30m': 0, '15m': 0<br />
    }<br />
<br />
}<br />

In [4]:
from backtesting.indicadores import squeeze_momentum_indicator
from scipy.stats import pearsonr

In [5]:
SMI_default = {}
for activo in activos:
    SMI_default[activo] = {}
    for t in temporalidades:
        df = data_raw[activo][t].iloc[:]
        df = squeeze_momentum_indicator(df)
        SMI_default[activo][t] = df.dropna()
        

In [6]:
SMI_default_corr = {}
for activo in activos:
    SMI_default_corr[activo] = {}
    for t in temporalidades:
        df = SMI_default[activo][t].iloc[:]
        df = df[['value', 'close']]
        SMI_default_corr[activo][t], _ = pearsonr(df['value'], df['close'])

print('Correlaciones')
for activo in activos:
    for t in temporalidades:
        print(f'{activo} {t} {round(SMI_default_corr[activo][t], 2)}')
    print()

Correlaciones
BTC 1d 0.36
BTC 4h 0.57
BTC 2h 0.57
BTC 1h 0.44
BTC 30m 0.2
BTC 15m 0.42

ETH 1d 0.41
ETH 4h 0.48
ETH 2h 0.58
ETH 1h 0.51
ETH 30m 0.15
ETH 15m 0.29

XRP 1d 0.41
XRP 4h 0.33
XRP 2h 0.42
XRP 1h 0.25
XRP 30m -0.03
XRP 15m 0.41

ENS 1d 0.44
ENS 4h 0.39
ENS 2h 0.43
ENS 1h 0.31
ENS 30m 0.04
ENS 15m 0.39



In [7]:
temporalidades_filtradas = {}
for activo in activos:
    temporalidades_filtradas[activo] = {}
    for t in temporalidades:
        if SMI_default_corr[activo][t] > 0.5:
            temporalidades_filtradas[activo][t] = SMI_default_corr[activo][t]

print('Temporalidades filtradas')
for activo in temporalidades_filtradas:
    for t in temporalidades_filtradas[activo]:
        print(f'correlacion {activo} {t}: {round(temporalidades_filtradas[activo][t], 2)}')
    print()

Temporalidades filtradas
correlacion BTC 4h: 0.57
correlacion BTC 2h: 0.57

correlacion ETH 2h: 0.58
correlacion ETH 1h: 0.51





### conclusiones 1:

- Si optimizo los parámetros para 1d, como afectan las demás temporalidades?
- Si optimizo los parámetros para una temporalidad con alta correlación, como afectan las demás temporalidades?
- Que diferencia existe entre aplicar la estrategia con parámetros por default y optimizados?

# Aplicar las condiciones de la estrategia a las temporalidades más correlacionadas y visualizar las últimas entradas

In [8]:
from backtesting.indicadores import backtest_entry
import plotly.graph_objects as go

In [9]:
entradas_backtest = {}
for activos in temporalidades_filtradas:
    for t in temporalidades_filtradas[activos]:
        entrada_short, entrada_long = backtest_entry(SMI_default[activos][t])
        dict_key = activos + '_' + t
        if len(entrada_short) > 0:
            dict_key_2 = dict_key + '_short'
            entradas_backtest.update({dict_key_2: entrada_short})
        if len(entrada_long) > 0:
            dict_key_2 = dict_key + '_long'
            entradas_backtest.update({dict_key_2: entrada_long})
        

In [10]:
## Obtenemos los dataframes de los activos y temporalidades filtradas, con indicador aplicado y parámetros default
## Visualizar las entradas generadas

ultimas_velas = 100

for activo in SMI_default:
    if len(temporalidades_filtradas[activo]) > 0:
        for t in temporalidades_filtradas[activo]:
            # print(f'{activo} {t}')
            # display(SMI_default[activo][t])
            # print()
            df = SMI_default[activo][t].iloc[-ultimas_velas:]

            candlestick = go.Candlestick(x=df.index,
                            open=df['open'],
                            high=df['high'],
                            low=df['low'],
                            close=df['close'])

            upper_band = go.Scatter(x=df.index, y=df['upper_BB'], name='upper_band', line=dict(color='red', width=1))
            lower_band = go.Scatter(x=df.index, y=df['lower_BB'], name='lower_band', line=dict(color='red', width=1))
            
            upper_KC = go.Scatter(x=df.index, y=df['upper_KC'], name='upper_KC', line=dict(color='blue', width=1, dash='dash'))
            lower_KC = go.Scatter(x=df.index, y=df['lower_KC'], name='lower_KC', line=dict(color='blue', width=1, dash='dash'))
            
            bar = go.Bar(x=df.index, y=df['value'], name='squeeze')


            fig = go.Figure(data=[candlestick, upper_band, lower_band, upper_KC, lower_KC])
            fig2 = go.Figure(data=[bar])
            
            fig.layout.xaxis.type = 'category'
            fig.layout.xaxis.rangeslider.visible = False
            # update figure 1
            fig.update_layout(
                autosize=False,
                width=1500,
                height=600,
                margin=dict(
                    
                    l=50,
                    r=50,
                    b=100,
                    t=50,
                    pad=4
                ),
                paper_bgcolor="LightSteelBlue",
                title=f'{activo} {t}'
            )
            # update figure 2
            fig2.update_layout(
                autosize=False,
                width=1500,
                height=300,
                margin=dict(
                    
                    l=50,
                    r=150,
                    b=1,
                    t=10,
                    pad=4
                ),
                paper_bgcolor="LightSteelBlue",
            )

            for k in entradas_backtest:
                if k.startswith(f'{activo}_{t}') and k.endswith('short'):
                    for x_value in entradas_backtest[f'{activo}_{t}_short'].index:
                        if x_value in df.index:
                            fig.add_shape(
                                type="line",
                                x0=x_value,
                                y0=df['low'].min(),
                                x1=x_value,
                                y1=df['high'].max(),
                                line=dict(color="red", width=1, dash="dash"),
                            )
                            fig2.add_shape(
                                type="line",
                                x0=x_value,
                                y0=df['value'].min(),
                                x1=x_value,
                                y1=df['value'].max(),
                                line=dict(color="red", width=1, dash="dash"),
                            )

                if k.startswith(f'{activo}_{t}') and k.endswith('long'):
                    for x_value in entradas_backtest[f'{activo}_{t}_long'].index:
                        if x_value in df.index:
                            fig.add_shape(
                                type="line",
                                x0=x_value,
                                y0=df['low'].min(),
                                x1=x_value,
                                y1=df['high'].max(),
                                line=dict(color="green", width=1, dash="dash"),
                            )
                            fig2.add_shape(
                                type="line",
                                x0=x_value,
                                y0=df['value'].min(),
                                x1=x_value,
                                y1=df['value'].max(),
                                line=dict(color="green", width=1, dash="dash"),
                            )

            fig.show()
            fig2.show()            

# Optimizar los parámetros del SMI para la temporalidad DIARIA y verificar la correlacion en las otras temporalidades

SMI_optimized = {<br />
    'BTC': <br />
    {'1d': pd.DataFrame()},<br />
    'ETH': <br />
    {'1d': pd.DataFrame()},<br />
    'XRP':<br />
    {'1d': pd.DataFrame()}<br />
<br />
}<br />

In [11]:
import numpy as np
from tqdm import tqdm
from copy import copy

import warnings
warnings.filterwarnings("ignore")

In [12]:
SMI_optimized = {}
SMI_optimized_corr = {}

length_range = range(18, 23)
mult_range = np.arange(1.5, 2.6, 0.1)
length_KC_range = range(18, 23)
mult_KC_range = np.arange(1, 2.1, 0.1)


for activo in data_raw:
    df = data_raw[activo]['1d'].copy()
    best_corr = -1
    best_params = None

    
    for length in length_range:
        for mult in mult_range:
            for length_KC in length_KC_range:
                for mult_KC in tqdm(mult_KC_range):
                    # Aplicar la función squeeze_momentum_indicator con los parámetros actuales
                    params = (length, mult, length_KC, mult_KC)
                    df = squeeze_momentum_indicator(df, params[0], params[1], params[2], params[3], length_atr=10, use_EMA=True)
                    df = df.dropna()
                    try:
                        corr, _ = pearsonr(df['value'], df['close'])
                        # Actualizar los mejores parámetros y coeficiente de correlación si se encuentra una mejora
                        if corr > best_corr:
                            best_corr = copy(corr)
                            best_params = copy(params)
                            
                            SMI_optimized.update({activo: df})
                            SMI_optimized_corr.update({f'{activo}_best_corr': best_corr, f'{activo}_best_params': best_params})
                    
                    except:
                        continue
                    


100%|██████████| 11/11 [00:00<00:00, 196.95it/s]


In [13]:
print('TEMPORALIDAD DIARIA')
print('Correlación parámetros por default')
for activo in SMI_default_corr:
    for t in SMI_default_corr[activo]:
        if t == '1d':
            print(f'{activo} 1d corr: {round(SMI_default_corr[activo][t], 2)}')

print()
print('Correlación parámetros optimizados')
for corr in SMI_optimized_corr:
    if corr.endswith('best_corr'):
        print(f'{corr[:-10]} 1d corr: {round(SMI_optimized_corr[corr], 2)}')


TEMPORALIDAD DIARIA
Correlación parámetros por default
BTC 1d corr: 0.36
ETH 1d corr: 0.41
XRP 1d corr: 0.41
ENS 1d corr: 0.44

Correlación parámetros optimizados
BTC 1d corr: 0.39
ETH 1d corr: 0.42
XRP 1d corr: 0.43
ENS 1d corr: 0.58


# Verificar como varía la correlación en las otras temporalidades

In [14]:
SMI_default_optimized = {}
for activo in data_raw:
    SMI_default_optimized[activo] = {}
    for t in temporalidades:
        df = data_raw[activo][t].iloc[:]
        label = f'{activo}_best_params'
        length, mult, length_KC, mult_KC = SMI_optimized_corr[label]
        # print(f'{activo}: params: {SMI_optimized_corr[label]}')
        df = squeeze_momentum_indicator(df, length, mult, length_KC, mult_KC, length_atr=10, use_EMA=True)
        
        SMI_default_optimized[activo][t] = df.dropna()

SMI_default_optimized_corr = {}
for activo in data_raw:
    SMI_default_optimized_corr[activo] = {}
    for t in temporalidades:
        df = SMI_default_optimized[activo][t].iloc[:]
        df = df[['value', 'close']]
        SMI_default_optimized_corr[activo][t], _ = pearsonr(df['value'], df['close'])

print('Correlaciones')
for activo in data_raw:
    for t in temporalidades:
        print(f'{activo} {t} ->  old: {round(SMI_default_corr[activo][t], 2)} | new: {round(SMI_default_optimized_corr[activo][t], 2)}')
    print()

Correlaciones
BTC 1d ->  old: 0.36 | new: 0.34
BTC 4h ->  old: 0.57 | new: 0.54
BTC 2h ->  old: 0.57 | new: 0.55
BTC 1h ->  old: 0.44 | new: 0.41
BTC 30m ->  old: 0.2 | new: 0.22
BTC 15m ->  old: 0.42 | new: 0.39

ETH 1d ->  old: 0.41 | new: 0.42
ETH 4h ->  old: 0.48 | new: 0.45
ETH 2h ->  old: 0.58 | new: 0.57
ETH 1h ->  old: 0.51 | new: 0.48
ETH 30m ->  old: 0.15 | new: 0.18
ETH 15m ->  old: 0.29 | new: 0.27

XRP 1d ->  old: 0.41 | new: 0.39
XRP 4h ->  old: 0.33 | new: 0.29
XRP 2h ->  old: 0.42 | new: 0.4
XRP 1h ->  old: 0.25 | new: 0.23
XRP 30m ->  old: -0.03 | new: 0.0
XRP 15m ->  old: 0.41 | new: 0.39

ENS 1d ->  old: 0.44 | new: 0.44
ENS 4h ->  old: 0.39 | new: 0.36
ENS 2h ->  old: 0.43 | new: 0.41
ENS 1h ->  old: 0.31 | new: 0.28
ENS 30m ->  old: 0.04 | new: 0.08
ENS 15m ->  old: 0.39 | new: 0.37



In [15]:
var1 = data_raw['BTC']['1d'].iloc[:]

var2 = squeeze_momentum_indicator(var1,18,1.5,18,1.1,10,True)

var2 = var2.dropna()

var3, _ = pearsonr(var2['value'], var2['close'])

var3

# np.arange(1, 2.1, 0.1)

0.34345662570710384

In [16]:
check = SMI_optimized['BTC'].dropna()
pearsonr(check['value'], check['close'])

PearsonRResult(statistic=0.3872895250459728, pvalue=8.3555921181237e-11)

In [17]:
# VERIFICAR QUE TAN LEJOS ESTÁN LOS PARÁMETROS OPTIMIZADOS DE A LOS LIMITES DEFINIDOS
# COMPROBAR SI UNA SEGUNDA INSTANCIA DE OPTIMIZACION MEJORA LA CORRELACION

# VERIFICAR QUE TAN LEJOS ESTÁN LOS PARÁMETROS OPTIMIZADOS DE A LOS LIMITES DEFINIDOS

default_params = (20, 2, 20, 1.5, 10, True)
SMI_optimized_corr['BTC_best_params']


(18, 1.5, 18, 1.1)

In [18]:
## TODO: Esto de acá fue una prueba y no es relevante para la función del módulo actual
import sys
sys.exit(0)

for señales in entradas_backtest:
    print(señales)
    display(entradas_backtest[señales].iloc[-1].to_frame().T)
    print()

SystemExit: 0

# Aplicar los criterios de la estrategia para cada configuración del indicador y visualizar las señales